In [1]:
%pip install sly

Note: you may need to restart the kernel to use updated packages.


In [4]:
from sly import Lexer
from sly import Parser


In [172]:
class CalcLexer(Lexer):
    # Set of token names. This is always required
    tokens = {
        'ID', 'EQUALS', 'LPAREN', 'RPAREN', 'COMMA', 'PLUS', 'DOT','LBRACKET', 'RBRACKET', 'TERNARY', 'NEW_LINE', 'COLON','AMPERSAND'
    }
    
    # String containing ignored characters between tokens
    ignore = ' \t'


    # Regular expression rules for tokens
    ID      = r'[a-zA-Z_][a-zA-Z0-9_]*'
    EQUALS  = r'='
    LPAREN  = r'\('
    RPAREN  = r'\)'
    COMMA   = r','
    PLUS    = r'\+'
    DOT     = r'\.'
    LBRACKET = r'\['
    RBRACKET = r'\]'
    TERNARY  = r'\?'
    NEW_LINE = r'\n'
    COLON = r':'
    AMPERSAND = r'&'

In [74]:
class Node:
    def __init__(self, type_, children=None, value=None):
        self.type = type_
        self.children = children
        self.value = value


In [177]:

from sly import Parser

class CalcParser(Parser):
    tokens = CalcLexer.tokens

    precedence = (
        ('left', 'PLUS'),
    )

    def __init__(self):
        self.names = {}


    @_('body')
    def start(self, p):
        return p.body
    
    @_('mapping_list NEW_LINE expression_list')
    def body(self,p):
        return Node('body', children=[p.mapping_list, p.expression_list])
    
    @_('mapping')
    def mapping_list(self, p):
        return Node('MappingList', children=[p.mapping])
    
    @_('mapping_list NEW_LINE mapping')
    def mapping_list(self, p):
        p.mapping_list.children.append(p.mapping)
        return p.mapping_list

    
    @_('AMPERSAND ID EQUALS ID')
    def mapping(self,p):
        return Node('Mapping', value=[p.ID0,p.ID1])
    
    @_('AMPERSAND ID EQUALS ID COLON ID')
    def mapping(self,p):
        return Node('Mapping', value=[p.ID0,p.ID1,p.ID2])
    
   

    @_('expression')
    def expression_list(self, p):
        return Node('ExpressionList', children=[p.expression])

    @_('expression_list PLUS expression')
    def expression_list(self, p):
        p.expression_list.children.append(p.expression)
        return p.expression_list

    @_('parallel_expr')
    def expression(self, p):
        return p.parallel_expr


    @_('command')
    def expression(self, p):
        return p.command

    @_('LPAREN expression_list RPAREN')
    def parallel_expr(self, p):
        return Node('ParallelBlock', children=[p.expression_list])
    
    @_('ID LPAREN param_list RPAREN')
    def command(self, p):
        return Node('Command', children=[Node('Id',value=p.ID), p.param_list])

    @_('ID LPAREN RPAREN')
    def command(self, p):
        return Node('Command', children=[Node('Id',value=p.ID)])

    @_('param')
    def param_list(self, p):
        return Node('ParamList',children=[p.param])

    @_('param_list COMMA param')
    def param_list(self, p):
        p.param_list.children.append(p.param)
        return p.param_list

    @_('ID EQUALS ID')
    def param(self, p):
        return Node('Param', children=[Node('Id',value=p.ID0), Node('ParamValue', value=[p.ID1])])

    @_('ID EQUALS param_value')
    def param(self, p):
        return Node('Param', children=[Node('Id',value=p.ID), p.param_value ])

    @_('ID DOT ID')
    def param_value(self, p):
        return Node('ParamValue', value=[p.ID0,p.ID1])

def print_tree(node, indent=0):
    if not node:
        return
    print('  ' * indent + str(node.type))
    if node.children:
        for child in node.children:
            print_tree(child, indent + 1)
    elif node.value is not None:
        print('  ' * (indent + 1) + str(node.value))
        if isinstance(node.value, Node):
            print_tree(node.value, indent + 1)


if __name__ == '__main__':
    lexer = CalcLexer()
    parser = CalcParser()
    #CommandPlan
    data = '(c1(p1=i1)) + (c2(p2=i2,p8=o1.m1) + (c3(p3=i4) + c4(p3=i6)))'
    data = f"""
    &c1=userdata:get_content
    &p1=file
    &i1=crendentials
    &c2=userdata:append
    &p2=file
    &i2=backup_credentials
    &m1=content
    &c3=userdata:append
    &p8=content
    &i3=Test
    &i4=Hello
    (c1(p1=i1)) + (c2(p1=i2,p8=o1.m1) + (c3(p1=i2,p8=i3) + c4(p1=i2,p8=i4)))"""
    result = parser.parse(lexer.tokenize(data))
    print_tree(result)

    
    
    



sly: Syntax error at line 1, token=NEW_LINE


AttributeError: No symbol ID3. Must be one of {AMPERSAND, ID0, EQUALS, ID1, COLON, ID2}.